# <center> MATH&ML-14. Рекомендательные системы. Часть I

### Юнит 5. Проблема холодного старта и popularity-based model

In [8]:
#Импорты

import pandas as pd
import numpy as np
import sklearn

In [9]:
ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv('data/movie.zip')
df=pd.merge(ratings, movies, how='left', on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


**Признаки в данных:**   
userId — id пользователя;   
movieId — id фильма;   
rating — выставленный пользователем рейтинг для фильма;   
timestamp — время выставления рейтинга;   
title — название фильма;   
genres — жанры, к которым относится фильм.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
 4   title      object 
 5   genres     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB


In [ ]:
#Задание 5.1
top_titles = df.title.value_counts(ascending=False)
display('Top-5:', top_titles.head(5))
display(top_titles.shape[0])

'Top-5:'

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

In [24]:
#Задание 5.2
sorted_by_rating = df.groupby('title')['rating'].mean().reset_index().sort_values(by=['rating', 'title'], ascending=[False, True])
#Отсечем фильмы с рейтингом ниже 5
sorted_by_rating[sorted_by_rating['rating'] == 5]

,title,rating
134,1971 (2014),5.0
397,A Blank on the Map (1971),5.0
414,A Gun for Jennifer (1997),5.0
430,A Night for Dying Tigers (2010),5.0
494,Abendland (2011),5.0
...,...,...
25607,Welcome to Australia (1999),5.0
25751,When I Walk (2013),5.0
25918,Who Killed Vincent Chin? (1987),5.0
26430,Year Zero: The Silent Death of Cambodia (1979),5.0


In [39]:
#Меняем подход. Отсечем фильмы, у которых менее 50 оценок
#Задание 5.3
films_df = df.title.value_counts().reset_index()
more_than_50_rates = films_df[films_df['count']>50]
more_than_50_rates = more_than_50_rates.rename(columns={'count': 'rates_count'})
print('Фильмов, у которых более 50 оценок:', more_than_50_rates.shape[0])

Фильмов, у которых более 50 оценок: 10472


In [47]:
#Задание 5.4
rec_list = pd.merge(more_than_50_rates, sorted_by_rating, on='title', how='left')
rec_list = rec_list.sort_values(by=['rating', 'title'], ascending=[False, True])
display(rec_list)

display('Рекомендованый фильм:', rec_list[:1])

,title,rates_count,rating
2,"Shawshank Redemption, The (1994)",63366,4.446990
28,"Godfather, The (1972)",41355,4.364732
14,"Usual Suspects, The (1995)",47006,4.334372
9,Schindler's List (1993),50054,4.310175
78,"Godfather: Part II, The (1974)",27398,4.275641
...,...,...,...
4900,Barney's Great Adventure (1998),419,1.163484
3851,Glitter (2001),685,1.124088
7012,Bratz: The Movie (2007),180,1.105556
4864,From Justin to Kelly (2003),426,0.973005


'Рекомендованый фильм:'

,title,rates_count,rating
2,"Shawshank Redemption, The (1994)",63366,4.44699
